In [4]:
import pandas as pd
from pathlib import Path
import awoc
from pprint import pprint
print("Setup complete")

Setup complete


In [5]:
#Loading csv file
df = pd.read_csv("C:\Program Files\Git\My-python-projects\project 1\world-data-2023.csv")

#Creation of a folder for all the CSV files that will be created throughout the code
file_dir = Path("C:\Program Files\Git\My-python-projects\project 1\Creation of new CSV files with the world-data-2023.csv sorted by continent") / "CSV Folder"
file_dir.mkdir(exist_ok=True)


df[df["Country"].str.startswith("The")] #Identify the extra charaters
df["Country"] = df["Country"].str.replace('The\s+','',regex=True) #Removing the extra charaters
df["Country"] = df["Country"].str.replace('Eswatini','Swaziland',regex=True)
new_df = df[~df['Country'].isin(['Vatican City', 'Republic of Ireland', 'Federated States of Micronesia', "North Macedonia","Palestinian National Authority","S�����������"])] #Creation of a new data frame without those values in it

data_frame = new_df.copy() #Making a copy of the data frame

#Crating a dictionnary that associates each country of the new data frame to his continent
dict_list = {}
my_world = awoc.AWOC()
for each_country in new_df["Country"]:
    dict_list[each_country] = my_world.get_country_data(each_country)['Continent Name']

data_frame['Continent'] = data_frame['Country'].map(dict_list) #Mapping the Country column with Continent one

data_frame_sorted = data_frame.sort_values(["Continent"])
#Remove unuseful values to the data frame
data_frame_sorted.drop(["Calling Code", "Capital/Major City", "Land Area(Km2)", "Armed Forces size", "Physicians per thousand", "Latitude", "Longitude", "Out of pocket health expenditure", "Largest city", "Co2-Emissions", "CPI", 'CPI Change (%)', 'Currency-Code', "Gross primary education enrollment (%)", 'Gross tertiary education enrollment (%)', 'Official language', 'Forested Area (%)', 'Tax revenue (%)', "Density\r\n(P/Km2)"], axis=1, inplace=True)

#Creating CSV files for each continent with all the countries include in this continent
for each_continent in my_world.get_continents_list():
    data_frame_sorted[data_frame_sorted['Continent'] == each_continent].sort_values(['Country']).to_csv(file_dir/f"{each_continent}.csv", index=False)

#Removing special characters in the values and convert them to float numbers
data_frame_sorted['Population'] = data_frame_sorted['Population'].str.replace(',','').astype(float)
data_frame_sorted['Urban_population'] = data_frame_sorted['Urban_population'].str.replace(',','').astype(float)
data_frame_sorted["GDP"] = data_frame_sorted["GDP"].str.replace('[\$,]', '', regex=True).astype(float)
data_frame_sorted["Minimum wage"] = data_frame_sorted['Minimum wage'].str.replace('$','').astype(float)
data_frame_sorted["Unemployment rate"] = data_frame_sorted["Unemployment rate"].str.replace('%', '', regex=True).astype(float)
data_frame_sorted["Agricultural Land( %)"] = data_frame_sorted["Agricultural Land( %)"].str.replace('%','', regex=True).astype(float)

aggregation_functions = {'GDP':'sum', 
                         'Population':'sum',
                         'Agricultural Land( %)':'mean',
                         'Birth Rate':'mean',
                         'Infant mortality':'mean', 
                         'Life expectancy':'mean', 
                         'Minimum wage':'mean',
                         'Unemployment rate':'mean', 
                         'Urban_population':'sum'
                         }
data_frame_groupped = data_frame_sorted.groupby(['Continent']).agg(aggregation_functions).round(2).reset_index()
data_frame_groupped.rename(columns={'Birth Rate':'Birth Rate_mean', "Infant mortality":'Infant mortality_mean', 'Life expectancy':'Life expectancy_mean', 'Population':'Population_sum', 'Urban_population':"Urban_population_sum", 'GDP':'GDP_sum ($)', "Minimum wage":"Minimum wage_mean ($)", 'Unemployment rate':'Unemployment rate_mean (%)', 'Agricultural Land( %)':'Agricultural Land( %)_mean (%)'})
data_frame_groupped_transpose = data_frame_groupped.T.reset_index()
data_frame_groupped_transpose.to_csv(file_dir/'Summarized_data.csv', index=False)
data_frame_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189 entries, 96 to 20
Data columns (total 17 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    189 non-null    object 
 1   Abbreviation                               186 non-null    object 
 2   Agricultural Land( %)                      185 non-null    float64
 3   Birth Rate                                 186 non-null    float64
 4   Fertility Rate                             185 non-null    float64
 5   Gasoline Price                             174 non-null    object 
 6   GDP                                        189 non-null    float64
 7   Infant mortality                           186 non-null    float64
 8   Life expectancy                            184 non-null    float64
 9   Maternal mortality ratio                   178 non-null    float64
 10  Minimum wage                   